In [ ]:
# Want to try making an interactive Folium map - need to convert the coordinates from BNG to WGS84
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.read_csv("../data/TellusSW_RAD_ALL.csv")

gdf = gpd.GeoDataFrame(
    df.copy(),
    geometry=[Point(xy) for xy in zip(df["X_BNG"],df["Y_BNG"])],
    crs="EPSG:27700"
).to_crs(epsg=4326)

gdf

In [ ]:
import folium
from folium.plugins import HeatMap

center = [gdf.geometry.y.median(), gdf.geometry.x.median()]
m = folium.Map(location=center, zoom_start=10, tiles="cartodbpositron")
df = pd.read_csv("../data/TellusSW_RAD_ALL.csv")
channels = {
    "processed_total_count": "Total Gamma",
    "processed_potassium": "Potassium (K-40)",
    "processed_uranium": "U-Series (Bi-214)",
    "processed_thorium": "Th-Series (Tl-208)"
}

initial_layer = "processed_total_count"

for col, label in channels.items():
    group = folium.FeatureGroup(name=label, show=(col == initial_layer))

    heat_data = [
        [row.geometry.y, row.geometry.x, row[col]]
        for _, row in gdf.iterrows()
    ]

    HeatMap(
        heat_data,
        radius=12,
        blur=18,
        max_zoom=12,
    ).add_to(group)

    group.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m.save("../output/interactive_radiometrics_heatmap.html")
# You can also view this as a webpage, as seen in the output folder.

m

# Hotspots correlate very strongly with areas high in granite, which seems to be the primary source of gamma radiation in these bands.
# The embedded map is too large for github so you'll have to run it yourself!